In [1]:
%reload_ext autoreload
%autoreload 2

import os
import sys
import importlib
import time

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from astropy import units as u
import h5py
from scipy.optimize import curve_fit

import utils as ut

utils_fold = "C:/Users/Pere/Documents/code/utils/"
sys.path.append(utils_fold)
import mutils as mut
from mutils.plot_utils import Figure
from mutils.plot_utils import Figure3D
importlib.reload(mut)

<module 'mutils' from 'C:\\Users/Pere/Documents/code/utils\\mutils\\__init__.py'>

We focus only on the PartType1, the highest resolution part of the simulation

In [2]:
simulation_name = 'galaxy_cluster'
output_folder = 'output'
snaps = ut.get_snapshots(f'../data/{simulation_name}/{output_folder}/')
files = [h5py.File(snap, 'r') for snap in snaps]
times = [file['Header'].attrs['Time'] for file in files]
zs = [file['Header'].attrs['Redshift'] for file in files]
_, N1, N2, N3 = files[0]['Header'].attrs['NumPart_ThisFile']
M1 = files[0]['Header'].attrs['MassTable'][1] # rest are zero
# M2_list_arr = [np.array(file['PartType2']['Masses']) for file in files]
# M3_list_arr = [np.array(file['PartType3']['Masses']) for file in files]

In [4]:
time_steps = [0.0, 0.5, 1.]

J = len(time_steps)
subplots = (1, J)
fig_size = 1920
ratio = 3
ts = 2
Fig = Figure(fig_size=fig_size, subplots=subplots, 
             ratio=ratio, theme='default', 
             wspace=0.025, hspace=0.05, grid=True,
             ts=ts)
fs = Fig.fs
axes = Fig.axes
axes_flat = Fig.axes_flat

resolution = 100
lim = 19

for i, time in enumerate(time_steps):

    ax = axes[0][i]

    idx = np.argmin(np.abs(np.array(times) - time))
    file = files[idx]
    PM = M1 * ut.M_gadget.to(u.Msun).value
    dh = 2*lim/resolution

    time_ = times[idx] * ut.T_gadget.to(u.Gyr).value
    z_ = zs[idx]

    positions = np.array(file['PartType1']['Coordinates']) / 1e3

    ranges = [(-lim, lim), (-lim, lim), (-lim, lim)]

    hist, edges = np.histogramdd(positions[:, :], bins=[resolution]*3, range=ranges)
    hist = hist * PM / dh**2

    projection = np.log(np.sum(hist, axis=2))

    ax.imshow((projection.T)[::-1,:], cmap='gnuplot', 
                extent=[-lim, lim, -lim, lim],
                vmin=25, vmax=34
                )
    
    if i == J-1:
        # add colorbar to ax
        cax = ax.inset_axes([1.05, 0.01, 0.075, 0.975])
        cax.tick_params(labelsize=ts*fs,
                        size=0.5*fs, width=0.15*fs,
                        pad=0.1*fs)
        cbar = plt.colorbar(cax=cax, mappable=ax.images[0], orientation='vertical')
        cbar.set_label(r'$\log \sigma$ [M$_\odot$ Mpc$^{-2} \, h^{2}$]', fontsize=fs*ts)

    if i == 0:
        ax.set_ylabel('y [Mpc $h^{-1}$]', fontsize=fs*ts)
    else:
        ax.tick_params(axis='y', labelleft=False)

    ax.set_xlabel('x [Mpc $h^{-1}$]', fontsize=fs*ts)

    ax.set_title(f'$z={z_:0.2f}, \ t={time_:0.2f}$ Gyr', fontsize=fs*ts,
                    pad=2*fs)

for ax in axes_flat:
    ax.set_facecolor('black')
    ax.grid(color='white',alpha=0.2)

bbox_inches = 'tight'
savepath = f'../figures/{simulation_name}/figure_1.jpg'
Fig.save(savepath, bbox_inches=bbox_inches)
plt.close()


C:\Users\Pere\AppData\Local\Temp\ipykernel_7068\1937750374.py:38: RuntimeWarning: divide by zero encountered in log
  projection = np.log(np.sum(hist, axis=2))


In [5]:
lims = [None, 40, 30, 20, 10, 5, 2]

rang = 20
bins = 100
rs = np.linspace(0, rang, bins)

mass = np.zeros(bins-1)
file = files[-1]
PM = M1 * ut.M_gadget.to(u.Msun).value
positions = np.array(file['PartType1']['Coordinates']) / 1e3
cm = ut.compute_recursive_cm(positions, lims=lims)
r = np.linalg.norm(positions-cm, axis=1)

hist, edges = np.histogram(r, bins=bins, range=(0, rang), density=False)
rs_ = (edges[:-1] + edges[1:]) / 2
dr = edges[1] - edges[0]
vol_el = 4*np.pi*(edges[1:]**3 - edges[:-1]**3) / 3
density = hist / vol_el * PM

for j in range(len(rs)-1):
    n = np.sum(r < rs[j])
    mass[j] = n * PM


rho_guess = mass[-1] / (4/3 * np.pi * r[-10]**3)
r_guess = r[-1]

popt, pcov = curve_fit(ut.NFW_mass_profile, rs[:-1], mass, p0=[rho_guess, r_guess])
print(popt)

nfw_profile_arr = ut.NFW_density_profile(edges[:-1], *popt)

ts = 2.5
Fig = Figure(fig_size=1080, subplots=(2,1), ratio=1.5, theme='default', ts=ts, height_ratios=[1, 0.4], hspace=0.)
ax = Fig.axes_flat[0]
ax2 = Fig.axes_flat[1]
fs = Fig.fs
lw = 0.4
leg_el_1, leg_el_2 =  [], []
labels1 = ['$\\rho_{\\text{rdm}}$ ($z=0$)', '$\\rho_{\\text{grid}}$ ($z=0$)',]
labels2 = ['NFW fit $(\\rho_{\\text{rdm}})$', 'NFW fit $(\\rho_{\\text{grid}})$',]

ax.plot(edges[:-1], nfw_profile_arr, lw=lw*fs, c='dimgrey', ls='dotted', label=r'$\rho_{\text{fit}}$')
ax.plot(edges[:-1], density[:], lw=lw*fs, c='k', ls='-', label=r'$\rho$')

residual = (density - nfw_profile_arr)/nfw_profile_arr
line2, = ax2.plot(edges[:-1], residual, lw=lw*fs, c='k', ls='-')


x0, y0 = 0.025, 0.15
dx, dy = .3, .13
num1 = ut.format_scientific(popt[0], 2)
ax.text(x0, y0, f'$\\rho_s =$ {num1}',
        fontsize=fs*ts, ha='left', va='center', transform=ax.transAxes, color='k')

num2 = ut.format_scientific(popt[1], 2)
ax.text(x0, y0 - dy, f'$r_s =$ {num2}',
        fontsize=fs*ts, ha='left', va='bottom', transform=ax.transAxes, color='k')


sc=1.1
ax.set_ylabel(r'$\log \rho$ [$\text{M}_{\! \odot} \, \text{Mpc}^{-3}$]', fontsize=fs*ts*sc)
ax2.set_ylabel(r'$(\rho - \rho_{\text{fit}})/\rho_{\text{fit}}$', fontsize=fs*ts*sc)
ax2.set_xlabel('$r$ [kpc]', fontsize=fs*ts*sc)

for ax_ in [ax, ax2]:
    ax_.set_xlim(edges[1], edges[-1])
    ax.set_yscale('log')
    ax_.set_xscale('log')    
ax.tick_params(axis='x', labelbottom=False)

# set legend in ax
ax.legend(loc='upper right', fontsize=fs*ts, frameon=True)

bbox_inches = 'tight'
savepath = f'../figures/{simulation_name}/figure_2.jpg'
Fig.save(savepath, bbox_inches=bbox_inches)
plt.close()

[3.74243732e+14 4.98684705e-01]


c:\Users\Pere\Documents\ULL\subjects\q2\tecnicas_simulacion_numerica\gadget4_project\code\utils.py:80: RuntimeWarning: divide by zero encountered in divide
  return rho_s / (x * (1 + x)**2)
C:\Users\Pere\AppData\Local\Temp\ipykernel_34648\2925416913.py:46: RuntimeWarning: invalid value encountered in divide
  residual = (density - nfw_profile_arr)/nfw_profile_arr
